The folk at [Code for Berlin](https://www.codefor.de/berlin/) have created a REST API offering access to the database of Berlin street trees and have [an issue open](https://github.com/codeforberlin/tickets/issues/3) asking people to try to do "something" with it. It seemed a cool way to look more deeply into the architecture of REST APIs on both the client and server side as well as playing with an interesting dataset, given I live in Berlin and like trees.

The API itself is built using the [Django REST Framework](https://www.django-rest-framework.org/) and is hosted [here](https://github.com/codeforberlin/trees-api-v2). An [interactive map](https://trees.codefor.de/) exists which uses the api to plot all the trees and allows some simple filtering on top of tiles from Open Street Map. I took a look and it proved a great intro to the data I wanted to do a deeper analysis of the data.

Some of the things I wanted to look into were:

* Which areas have the most trees, the oldest trees etc
* Are there any connections between the number of trees and other datapoints (air quality, socioeconomic demographics etc)
* Why are there no trees showing on my street even though I can see some out the window as I type this? 

## What sort of data is there and how can it be consumed? 

One of the cool things about the Django REST Framework is the way it's API can be explored out of the box. Simply point your browser to the API using the following link:

https://trees.codefor.de/api/v2

You should see something like this:

```

HTTP 200 OK
Allow: GET, HEAD, OPTIONS
Content-Type: application/json
Vary: Accept

{
    "trees": "https://trees.codefor.de/api/v2/trees/",
    "species": "https://trees.codefor.de/api/v2/species/",
    "genera": "https://trees.codefor.de/api/v2/genera/",
    "boroughs": "https://trees.codefor.de/api/v2/boroughs/"
}

```

Essetially this is telling us that we have four endpoints - trees, species, genera and boroughs. You can follow the links to each one to get more details. To explore the data available, I hacked together a simple python wrapper which you can find here: 

https://github.com/scrambldchannel/berlin-trees-api-pywrapper

### Usage

The wrapper can be installed via pip:

```
pip install git+https://github.com/scrambldchannel/berlin-trees-api-pywrapper.git
```

#### Setup the wrapper

Note I am specifying version 2. When I look at the

In [1]:
# Import the module and other useful libs

import json
from berlintreesapiwrapper import TreesWrapper

# Instantiate the api wrapper object
# you can change the base url if you are running a local instance of the api 

base_url = "https://trees.codefor.de/api/"
api_version = 2
api = TreesWrapper(api_root = base_url, version = api_version)

#### Calling functions

There is a function defined for each endpoint. At this stage, each function accepts only a couple of parameters. Each endpoint returns paginated results (the current config seems to return ten results per page) so the page number is a valid parameter for each function, defaulting to 1 if not supplied. See examples below.   

#### Trees endpoint

The most versatile endpoint is the trees endpoint which returns sets of individual trees. The endpoint allows filtering in a number of different ways (see https://github.com/codeforberlin/trees-api-v2#making-queries).

My basic wrapper function doesn't support anything other than a simple dump of all trees, by page, at this stage. This was sufficient for pulling all the data but I will look into enhancing this wrapper later, the ability to filter trees based on location is particular interesting. 

```python
# Eg. request first page of all trees

ret_trees = api.get_trees()

# Eg. request the 5000th page of all trees

ret_trees = api.get_trees(page=5000)
```

#### Other endpoints

The other endpoints just return a count of the trees by borough, species and genus. Results can be filtered by page and the name of the borough etc. See examples below.

```python
# Eg. request first page of the borough count

ret_borough = api.get_boroughs()

# Eg. request the count for a specific borough

ret_borough = api.get_boroughs(borough = "Friedrichshain-Kreuzberg")

# Eg. request the count for a specific species

ret_species = api.get_species(species = "Fagus sylvatica")

# Eg. request a specific page of the count of genera

ret_genera = api.get_genera(page = 13)
```



## Data exploration

First, I need to get the data into a format I can analyse it easily. 

### Look at structure for a single tree

I want to pull it all individual trees into a single dataframe. To do so, I returned to the trees endpoint. The relevant part of the json result is contained within "features" and an individual tree looks like this:

```json

{
    "geometry": {
        "coordinates": [
            13.357809221770479,
            52.56657685261005
        ],
        "type": "Point"
    },
    "id": 38140,
    "properties": {
        "age": 80,
        "borough": "Reinickendorf",
        "circumference": 251,
        "created": "2018-11-11T12:22:35.506000Z",
        "feature_name": "s_wfs_baumbestand_an",
        "genus": "ACER",
        "height": 20,
        "identifier": "s_wfs_baumbestand_an.7329",
        "species": "Acer pseudoplatanus",
        "updated": "2018-11-11T12:22:35.506000Z",
        "year": 1938
    },
    "type": "Feature"
}
```

### Write script to pull all trees

Essentially I want to pull all of these trees into a single dataframe by iterating over every page of the trees endpoint. I hacked together this code to accomplish this. It also converted the result to a geodataframe based on the long/lat information returned. Note, this was really slow, probably wasn't the best way to do it and there are other ways of sourcing the raw data. That said, I wanted to do it as a PoC.

```python
# loop over the pages until we reach the end and append the values we're interested to lists

while True:
    this_page = api.get_trees(page=page).json()
    next_page = this_page["next"]
    for row in range(len(this_page['features'])):
        ids.append(this_page['features'][row]['id'])
        age.append(this_page['features'][row]['properties']['age'])
        borough.append(this_page['features'][row]['properties']['borough'])
        circumference.append(this_page['features'][row]['properties']['circumference'])
        genus.append(this_page['features'][row]['properties']['genus'])
        height.append(this_page['features'][row]['properties']['height'])
        species.append(this_page['features'][row]['properties']['species'])
        year.append(this_page['features'][row]['properties']['year'])        
        lat.append(this_page['features'][row]['geometry']['coordinates'][0])
        long.append(this_page['features'][row]['geometry']['coordinates'][1])        

    page = page + 1
    
    if(next_page) is None:
        break

# create dataframe from resulting lists 

df = pd.DataFrame(
    {'id': ids,
    'age' : age,
    'borough' : borough,
    'circumference' : circumference,
    'genus' : genus,
    'height' : height,
    'species' : species,
    'year': year,
    'Latitude': lat,
    'Longitude': long})
```

After running once, I saved it to a csv for future analysis. As an aside, I've recently started using the amazing [VisiData](https://visidata.org/) for this sort of analysis of data in text form but have done it here using Pandas. 

### Load into Pandas dataframe

In [77]:
# Import ilbraries

import numpy as np
import pandas as pd
import geopandas as gpd

# load csv

dataset_path = '../datasets/'
df = pd.read_csv(filepath_or_buffer = dataset_path + 'all_trees.csv', index_col = 0, encoding='utf-8')

### Convert to Geopandas dataframe

Given we have lat/long for each tree, let's convert it to a Geopandas dataframe which might come in handy later.

In [78]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude)) 

### Get an overview of the data

This gives an overview of the data which is a useful starting point and helps give insight into data quality issues there might be.

#### This is what the data looks like

In [79]:
gdf.head()

,id,age,borough,circumference,genus,height,species,year,Latitude,Longitude,geometry
0,161995,0.0,Spandau,0,UNBEKANNT,0,Unbekannt,0,13.155043,52.495410,POINT (52.49541034680273 13.15504341871138)
1,161996,58.0,Spandau,132,BETULA,15,Betula pendula,1960,13.186265,52.503243,POINT (52.50324279543711 13.18626537619331)
2,161997,0.0,Spandau,0,UNBEKANNT,0,Unbekannt,0,13.153210,52.496383,POINT (52.49638311716633 13.15321021931674)
3,161998,0.0,Spandau,0,UNBEKANNT,0,Unbekannt,0,13.155034,52.495817,POINT (52.49581668236215 13.15503440704845)
4,161999,0.0,Spandau,0,UNBEKANNT,0,Unbekannt,0,13.154813,52.495896,POINT (52.49589600666037 13.15481312573682)


#### Get a row count

In [80]:
len(gdf.index)

268730

#### Use the describe method on the numeric fields

In [81]:
gdf.describe(percentiles = [.20, .40, .60, .80], include = [ 'float', 'int'])

,id,age,circumference,height,year,Latitude,Longitude
count,268730.000000,238081.000000,268730.000000,268730.000000,268730.000000,268730.000000,268730.000000
mean,134362.814204,28.812774,111.276482,10.168723,1169.399375,13.368672,52.521611
std,77512.112664,36.062176,81.262925,8.992996,967.919838,0.113482,0.062437
min,6.000000,0.000000,0.000000,0.000000,0.000000,13.036010,52.327146
20%,53314.800000,0.000000,51.000000,0.000000,0.000000,13.255557,52.454740
40%,107677.600000,0.000000,79.000000,7.000000,0.000000,13.343608,52.507712
50%,134664.500000,20.000000,94.000000,10.000000,1943.000000,13.394595,52.524516
60%,162326.400000,30.000000,113.000000,12.000000,1965.000000,13.424421,52.539060
80%,214635.200000,53.000000,160.000000,17.000000,1985.000000,13.470514,52.579623
max,268730.000000,2017.000000,12838.000000,1340.000000,19965.000000,13.738202,52.650461


##### A number of things stand out at a glance:

* All columns seem to be populated for all rows except age (ie their counts match the total row count)
* That said, all of the value columns have zeros so there are some gaps in the data
* The max values for all measures are clearly spurious based on the percentiles
* There must be some duplicates in the id column which I'd believed should be unique
* The age and the year (presuming it means the year of planting) should correspond however the percentiles don't reflect this
* The long/lat values don't seem to have an extreme outliers

#### Use the describe method on the string fields

In [82]:
gdf[['borough', 'genus', 'species']].describe()

,borough,genus,species
count,268730,268112,268123
unique,12,130,635
top,Pankow,ACER,Acer platanoides
freq,57035,54409,22522


##### Things to note:

* Population of the borough field is complete but genus and species have some gaps
* Perhaps there is a mix of upper/lower case that might need to be normalised



In [ ]:
### Try to address data quality

Let's try to either correct the outliers (if possible) or remove them from calculations by setting the values to NaN. For the circumference and height data, this is relatively straightforward, for the age / year numbers, it might be possible to derive one from the other. 

#### Setting 0s to NaN

Doing this should remove the 0s from the calculations while retaining any information that is available for that tree. 

In [64]:
gdf['age'].replace(0, np.nan,  inplace=True)
gdf['circumference'].replace(0, np.nan,  inplace=True)
gdf['height'].replace(0, np.nan,  inplace=True)
gdf['year'].replace(0, np.nan,  inplace=True)

#### Deriving age from year and vice versa

Let's check the assumption the age and year are connected, that is:


```
age = 2018 - year
```

Let's try to check that assumption, perhaps a bit of a hack but it does the trick. **There must be a better way to do this**

In [92]:
total = 0
for i in range(0,2020):
    count = gdf[abs(gdf.age) == (i - gdf.year)]['id'].count()     
    if count != 0:
        print(i, count)
        total = total + count
print(total)
        

0 100445
6 3
9 2
13 3
15 5
27 1
52 1
114 3
2014 3
2015 14
2016 1141
2017 100
2018 136360
238081


So there's a bit of variation but essentially, either the year is set 0 or the age is usually about equal to the number of years from the year. Let's just set the age to the 2018 - year

In [ ]:
gdf['age'].replace(0, np.nan,  inplace=True)

In [10]:
# Get oldest tree(s)

gdf[gdf['age'] == gdf['age'].max()]

,id,age,borough,circumference,genus,height,species,year,Latitude,Longitude,geometry
201767,166462,2017.0,Spandau,25.0,PRUNUS,6.0,Prunus avium,1,13.208645,52.538109,POINT (52.53810938066821 13.20864503275809)


In [11]:
# This seems to show that anything with a year has a sensible age

all_trees_gdf.loc[(all_trees_gdf['age'] == 0) & (all_trees_gdf['year'] >= 1) & (all_trees_gdf['year'] < 2018)]

NameError: name 'all_trees_gdf' is not defined

In [ ]:
# but there are a lot of missing ages that have years

all_trees_gdf.loc[(all_trees_gdf['age'].isnull()) & (all_trees_gdf['year'] >= 1) & (all_trees_gdf['year'] < 2018)]

In [ ]:
# What about circumference? 

all_trees_gdf.loc[(all_trees_gdf['circumference'] >= 500) & (all_trees_gdf['circumference'] <= 13000)]

In [ ]:
# this should give the oldest tree by  

all_trees_gdf.sort_values('age').drop_duplicates(['borough'], keep='last')[[]]

In [ ]:
# this will give you the tree with the highest cirucmference for each borough 

# more columns can be added to the list passed to drop_duplicates to effectively group by more columns

all_trees_gdf.sort_values('circumference').drop_duplicates(['borough'], keep='last').head()